# Метаклассы

## Объекты

Вспоминаем, что в питоне все является объектом. В том числе классы. 

In [ ]:
class MyClass: # этот код создает в памяти объект на который ссылается переменная MyClass
    def __init__(self, a):
        print('Created instance of MyClass!')
        self.a = a

Объект `MyClass` может сам порождать объекты --> является классом

In [ ]:
my_obj = MyClass(1)

Объект ```my_obj``` уже не может порождать объекты, потому что является экземпляром класса, но не классом!

При этом с классом можно делать все то же самое, что и с любым объектом:

In [ ]:
# записать в переменную
class_to_make = MyClass
my_obj = class_to_make(1)

In [ ]:
my_obj.a

In [ ]:
# передать в функцию 
def create_instance(class_object, a=1):
    print(a)
    return class_object(a)

In [ ]:
my_obj = create_instance(MyClass, 1)

In [ ]:
my_obj.a

In [ ]:
# добавить или изменить атрибут (это будет атрибут класса)
MyClass.new_class_attr = 10
my_obj1 = MyClass(1)

In [ ]:
my_obj1.new_class_attr

In [ ]:
MyClass.new_class_attr = 20

In [ ]:
my_obj1.new_class_attr

## Что происходит при создании экземпляра класса?

#### \_\_new\_\_ и \_\_init\_\_ 

![](https://i.stack.imgur.com/MgRbx.png)
+ `__new__()` - отвечает за создание нового экземпляра класса, возвращает новый объект (**должен быть return**)
+ `__init__()` - отвечает за инициализацию нового экземпляра класса - объявить какие у него есть атрибуты, какие у них значения (**без returna**)


In [ ]:
class MyClass:

    def __init__(self):
        self.my_attr = 1
        print('init called')

    def __new__(cls, *args, **kwargs):
        print('new called')
        return super().__new__(cls, *args, **kwargs)


In [ ]:
new_instance = MyClass()
print(new_instance.my_attr)

### Что такое меткласс?

   **Метакласс** - класс, экземпляры которого сами являются классами (могут порождать свои экземпляры). 
![](https://blog.ionelmc.ro/2015/02/09/understanding-python-metaclasses/instance-of.png)

Создаем метакласс, который будет порождать классы "студент Х-го курса":

In [ ]:
# переопределим конструктор, чтобы атрибут name передавался в качестве аргумента
def __init__(self, name):
    self.name = name

In [ ]:
# создадим метод introduce
def introduce(self):
    return "Hello I am %s, %s year student!" % (self.name, str(self.year))

In [ ]:
# допустим мы хотим отдельно передавать методы в виде списка
# и так, чтобы они автоматически добавлялись с нужным именем
student_methods = [introduce, __init__]

In [ ]:
class StudentMetaClass(type): # обязательно наследуемся от type 
    def __new__(cls, name, bases, attrs):
        for method in attrs['methods']:
            attrs[method.__name__] = method # добавляем пары ключ - название метода, значение -  метод
        attrs.pop('methods') # удаляем methods из словаря атрибутов 
        return super().__new__(cls, name, bases, attrs)

`type` на самом деле тоже является метаклассом, который Python внутренне использует для создания всех классов

In [ ]:
# показывает, экземпляром какого класса явлется объект
type(object)

In [ ]:
new_classes = []
for i in range(1,5):
    new_classes.append(StudentMetaClass('Student%sYear'%str(i), (object, ), {'year': i, 'methods': student_methods})) 

In [ ]:
names = ['Vasya', 'Masha', 'Petya', 'Dasha']
for i, class_ in enumerate(new_classes):
    obj = class_(name=names[i])
    print(obj)
    print(obj.year)
    print(obj.introduce()+'\n')

In [ ]:
type(obj)

In [ ]:
type(new_classes[-1])

### Аргумент metaclass

При написании класса можно добавить аргумент  metaclass, тогда питон при создании класса будет использовать указанный метакласс, а не type  
При указании metaclass питон 
+ перехватывает создание класса
+ изменяет класс
+ возвращает модифицированный объект класса

In [ ]:
class Student1Year(metaclass=StudentMetaClass):
    # задаем атрибуты, такие же как в словаре переданном StudentMetaClass последним аргументом
    year = 1 
    methods = student_methods

In [ ]:
student = Student1Year('Boris')

In [ ]:
student.introduce()

In [ ]:
student.year

In [ ]:
type(student)

In [ ]:
type(Student1Year)

### Что происходит при создании экземпляра класса (теперь еще и с метаклассами)?

+ ```__call__()``` - определяет поведение, когда экземпляр класса вызывают (как функцию - со скобочками)

In [ ]:
class MyClass:

    def __call__(cls, *args, **kwargs):
        print('called method call')

my_class_instance = MyClass()

In [ ]:
my_class_instance()

![](https://i.stack.imgur.com/YVB4Q.png)

In [ ]:
class MyMeta(type):
    def __new__(cls, *args, **kwargs):
        print('called new of metaclass')
        return super().__new__(cls, *args, **kwargs)

    def __call__(cls, *args, **kwargs):
        print('called call of metaclass')
        return super().__call__(*args, **kwargs)

class MyClass(metaclass=MyMeta):
    def __init__(self):
        self.my_attr = 1
        print('init called')

    def __new__(cls, *args, **kwargs):
        print('new called')
        return super().__new__(cls, *args, **kwargs)

# MyClass() - тот самый __call__, мы вызываем экземпляр класса MyMeta (объект MyClass) как функцию
# my_class = MyClass() 

**Задание**: 
   + написать метакласс, который переводит названия всех атрибутов и методов (кроме служебных) в верхний регистр
   + служебный = начинается и заканчивается на два нижих подчеркивания

In [ ]:
# пример работы
class MyClass(metaclass=UpperCaseMetaclass):
    attr1 = 1
    
    def method1(self):
        print('method 1')

In [ ]:
my_object = MyClass()
my_object.ATTR1
my_object.METHOD1()
# 1

## Зачем нужны метаклассы - примеры

### Синглтон

Паттерн синглтон:
+ создание одного и только одного экземпляра класса
+ предоставление глобальной точки доступа к нему

Пример в питоне - None:

In [ ]:
a = None

In [ ]:
b = None

In [ ]:
a is b

In [ ]:
print(id(a), id(b), id(None))

In [ ]:
# можно делать так и наследоваться от него
class SingletonBase:
    def __new__(cls):
        if not hasattr(cls, 'instance'): # проверяем что существует только один экземпляр
            cls.instance = super().__new__(cls) # сохраняем в атрибуте класса информацию о созданном экземпляре
        return cls.instance
s = SingletonBase()
print("Object created", s)
s1 = SingletonBase() # точка доступа через создание экземпляра
print("Object created", s1)

In [ ]:
s is s1

In [ ]:
class Singleton1(SingletonBase):
    pass

In [ ]:
class Singleton2(SingletonBase):
    pass

In [ ]:
s1 = Singleton1()
s1_also = Singleton1()

In [ ]:
s2 = Singleton2()
s2_also = Singleton2()

In [ ]:
s1 is s1_also

In [ ]:
# но
s1 is s2

In [ ]:
# но
print(type(s1))
print(type(s2))

Метакласс для создания синглтона:

In [ ]:
# можно создать метакласс
class MetaSingleton(type):
    _instances = {} # храним set созданных экземпляров всех классов 
    def __call__(cls, *args, **kwargs):
        if cls not in cls._instances:
            cls._instances[cls] = super().__call__(*args, **kwargs)
        return cls._instances[cls]
    
class Logger(metaclass=MetaSingleton):
    pass

class Something(metaclass=MetaSingleton):
    some_attr = 0

logger1 = Logger()
print(len(MetaSingleton._instances))
logger2 = Logger()
print(len(MetaSingleton._instances))
something1 = Something()
print(len(MetaSingleton._instances))
something2 = Something()
print(len(MetaSingleton._instances))

In [ ]:
print(logger1 is logger2)
print(something1 is something2)

In [ ]:
print(logger1 is something2) 

In [ ]:
print(type(logger1))
print(type(something1))

In [ ]:
class ParentLogger:
    is_parent = True

# наследуемся как обычно
class Logger(ParentLogger, metaclass=MetaSingleton):
    pass

In [ ]:
logger1 = Logger()

In [ ]:
Logger.__bases__

In [ ]:
type(logger1)

In [ ]:
type(Logger)

**Задание:**
+ выяснить и рассказать мне, наследуется ли принадлежность к метаклассу
+ то есть будут ли дочерние классы например Logger иметь тот же метакласс

### ORM

**Object Relational Mapping** - отображение отношений реляционной бд в классы и объекты
![](https://www.fullstackpython.com/img/visuals/orms-bridge.png)

Примеры:
+ SQLAlchemy
+ DjangoORM
+ SQLObject
+ ClickhouseORM
+ PonyORM
+ и т.д. 

Зачем нужно:
+ упрощает написание запросов и работу с их результатами
    ![](https://www.dropbox.com/s/em0e38etaqmopgr/Screenshot%20from%202021-01-20%2001-38-55.png?dl=1)
+ более абстрактный и универсальный интерфейс
+ проще дебажить и тестировать
+ многие вещи автоматизированы
